# ChatGithubCopilot

This will help you getting started with GithubCopilot [chat models](/docs/concepts/#chat-models). For detailed documentation of all ChatGithubCopilot features and configurations head to the [API reference](https://api.python.langchain.com/en/latest/chat_models/langchain_github_copilot.chat_models.ChatGithubCopilot.html).

## Overview
### Integration details

| Class | Package | Local | Serializable | JS support| Package downloads | Package latest |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| [ChatGithubCopilot](https://api.python.langchain.com/en/latest/chat_models/langchain_github_copilot.chat_models.ChatGithubCopilot.html) | [langchain-github-copilot](https://api.python.langchain.com/en/latest/github_copilot_api_reference.html) | ❌ | ❌ | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-github-copilot?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-github-copilot?style=flat-square&label=%20) |

### Model features
| [Tool calling](/docs/how_to/tool_calling) | [Structured output](/docs/how_to/structured_output/) | JSON mode | [Image input](/docs/how_to/multimodal_inputs/) | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | Native async | [Token usage](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | ✅ | ❌ | 

## Setup

To access GithubCopilot models you'll need to create a Github account, get a Github token, and install the `langchain-github-copilot` integration package.

### Credentials

Use the following function to set up your Github Token as an environment variable.

In [1]:
import requests, os, time

def setup():
    resp = requests.post('https://github.com/login/device/code', headers={
            'accept': 'application/json',
            'editor-version': 'Neovim/0.6.1',
            'editor-plugin-version': 'copilot.vim/1.16.0',
            'content-type': 'application/json',
            'user-agent': 'GithubCopilot/1.155.0',
            'accept-encoding': 'gzip,deflate,br'
        }, data='{"client_id":"Iv1.b507a08c87ecfe98","scope":"read:user"}')


    # Parse the response json, isolating the device_code, user_code, and verification_uri
    resp_json = resp.json()
    device_code = resp_json.get('device_code')
    user_code = resp_json.get('user_code')
    verification_uri = resp_json.get('verification_uri')

    # Print the user code and verification uri
    print(f'Please visit {verification_uri} and enter code {user_code} to authenticate.')


    while True:
        time.sleep(5)
        resp = requests.post('https://github.com/login/oauth/access_token', headers={
            'accept': 'application/json',
            'editor-version': 'Neovim/0.6.1',
            'editor-plugin-version': 'copilot.vim/1.16.0',
            'content-type': 'application/json',
            'user-agent': 'GithubCopilot/1.155.0',
            'accept-encoding': 'gzip,deflate,br'
            }, data=f'{{"client_id":"Iv1.b507a08c87ecfe98","device_code":"{device_code}","grant_type":"urn:ietf:params:oauth:grant-type:device_code"}}')

        # Parse the response json, isolating the access_token
        resp_json = resp.json()
        access_token = resp_json.get('access_token')

        if access_token:
            break
        
    os.environ['GITHUB_TOKEN'] = access_token

    print('Authentication success!')
    
setup()

Please visit https://github.com/login/device and enter code 2E12-05E0 to authenticate.
Authentication success!


If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [2]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

### Installation

The LangChain GithubCopilot integration lives in the `langchain-github-copilot` package:

In [3]:
%pip install -qU langchain-github-copilot

Note: you may need to restart the kernel to use updated packages.


## Instantiation

Now we can instantiate our model object and generate chat completions:

In [5]:
from langchain_github_copilot import ChatGithubCopilot

llm = ChatGithubCopilot(
    model='my-copilot',
    temperature=0.9,
    max_tokens=100,
    timeout= None,
    stop= ["<END>"]
)

## Invocation

In [6]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'model': 'gpt-3.5-turbo-0613'}, id='run-b99f23f6-467f-4885-a914-58bf6f068993-0', usage_metadata={'input_tokens': 31, 'output_tokens': 8, 'total_tokens': 39})

In [7]:
print(ai_msg.content)

J'adore la programmation.


## Chaining

We can [chain](/docs/how_to/sequence/) our model with a prompt template like so:

In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe das Programmieren.', additional_kwargs={}, response_metadata={'model': 'gpt-3.5-turbo-0613'}, id='run-c1f95c30-1aad-4deb-9a99-75e1512161e0-0', usage_metadata={'input_tokens': 26, 'output_tokens': 6, 'total_tokens': 32})

## API reference

For detailed documentation of all ChatGithubCopilot features and configurations head to the API reference: https://api.python.langchain.com/en/latest/chat_models/langchain_github_copilot.chat_models.ChatGithubCopilot.html